In [2]:
import fastexcel as fex
import polars as pl
import xlsxwriter

with open('data/excel data/singthai PUR FFB 2024.xlsx', 'rb') as f:
    data = f.read()
    excel = fex.read_excel(data)

df = pl.DataFrame()
columns = ['Date Out', 'Serial No', 'Lorry No', 'Supplier', 'Net Wt(Ton)', 'Price(ton)', 'Gross Amt', 'TPT CHRG', 'Tpt Amt', 'Worker Chrg', 'Worker Amt', 'Others', 'NET']
for sheet in excel.sheet_names:
    if 'FFB' in sheet:
        temp = pl.read_excel(data, sheet_name=sheet)
        temp = temp.drop_nulls(['Date Out']).select(columns)
        if temp['Date Out'].dtype != pl.Date:
            temp = temp.with_columns(
                pl.when(temp['Date Out'].str.contains(r"\d{4}-\d{2}-\d{2}"))
                .then(temp['Date Out'].str.to_date("%Y-%m-%d %H:%M:%S", strict=False))
                .otherwise(temp['Date Out'].str.to_date("%d/%m/%Y", strict=False))
                .alias("Date Out")
            )
        temp = temp.with_columns(
            pl.col("Net Wt(Ton)").cast(pl.Float32),
            pl.col("Price(ton)").cast(pl.Float32),
            pl.col("Gross Amt").cast(pl.Float32),
            pl.col("TPT CHRG").cast(pl.Float32),
            pl.col("Tpt Amt").cast(pl.Float32),
            pl.col("Worker Chrg").cast(pl.Float32),
            pl.col("Worker Amt").cast(pl.Float32),
            pl.col("Others").cast(pl.Float32),
            pl.col("NET").cast(pl.Float32)
        )
            
        df = pl.concat([df, temp])
df = df.with_columns(
    month = df['Date Out'].dt.month(),
).sort('month')
df

Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 13, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 16, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 11, falling back to string


Date Out,Serial No,Lorry No,Supplier,Net Wt(Ton),Price(ton),Gross Amt,TPT CHRG,Tpt Amt,Worker Chrg,Worker Amt,Others,NET,month
date,str,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8
2024-01-02,"""P0023888""","""8758""","""2001""",0.1,670.0,67.0,null,0.0,null,0.0,null,67.0,1
2024-01-04,"""P0023916""","""6689""","""2001""",0.46,655.0,301.299988,null,0.0,null,0.0,null,301.299988,1
2024-01-04,"""P0023920""","""6689""","""2001""",0.55,655.0,360.25,null,0.0,null,0.0,null,360.25,1
2024-01-04,"""P0023923""","""5087""","""2001""",0.72,655.0,471.600006,null,0.0,null,0.0,null,471.600006,1
2024-01-04,"""P0023925""","""5087""","""2001""",0.73,655.0,478.149994,null,0.0,null,0.0,null,478.149994,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-12-11,"""P0028596""","""JQH3813""","""2101""",1.09,1050.0,1144.5,40.0,43.599998,80.0,87.199997,null,1013.700012,12
2024-12-04,"""P0028538""","""JQH3813""","""2101B""",0.86,1050.0,903.0,30.0,25.799999,100.0,86.0,null,791.200012,12
2024-12-15,"""P0028644""","""NCR3737""","""2114""",2.92,1050.0,3066.0,15.0,43.799999,90.0,262.799988,null,2759.399902,12


In [108]:
import xlsxwriter
pivot = (df
    .drop(['Date Out', 'Serial No', 'Lorry No', 'Price(ton)', 'TPT CHRG', 'Worker Chrg', 'Others', 'NET'])
    .melt(id_vars=["Supplier", "month"], variable_name="Metric", value_name="Value")
    .pivot(
        on=["Metric"],
        index=["Supplier", "month"],
        values="Value",
        aggregate_function="sum"
    )
    .sort("Supplier", "month")
    .with_columns(
        pl.col('month').cast(pl.Utf8)
    )
)

df_subtotal = pivot.group_by("Supplier").agg(
    pl.sum('Net Wt(Ton)'),
    pl.sum('Gross Amt'),
    pl.sum('Tpt Amt'),
    pl.sum('Worker Amt'),
).with_columns(
    month = pl.lit("9999")
).select(
    pivot.columns
)

df_total = pivot.with_columns(
    pl.sum('Net Wt(Ton)'),
    pl.sum('Gross Amt'),
    pl.sum('Tpt Amt'),
    pl.sum('Worker Amt'),
    month = pl.lit("Total"),
    Supplier = pl.lit("Total")
).head(1).select(pivot.columns)

concat = (pl
    .concat([pivot, df_subtotal, df_total])
    .with_columns(
        month = pl.col('month').cast(pl.Int32, strict=False),
    )
    .sort('Supplier', 'month')
    .with_columns(
        pl.col('Net Wt(Ton)').round(2),
        pl.col('Gross Amt').round(2),
        pl.col('Tpt Amt').round(2),
        pl.col('Worker Amt').round(2),
        pl.col('month').cast(pl.Utf8).str.replace("9999", "total"),
    )
)

file_path = 'data/export/2024summary3.xlsx'

# Open an Excel writer using xlsxwriter
with xlsxwriter.Workbook(file_path) as writer:
    worksheet = writer.add_worksheet("Summary")

    heading_format = writer.add_format({'bold': True, 'font_color': '#050505', 'bg_color': '#71b07c'})
    format_total = writer.add_format({'bg_color': '#bbedc4', 'font_color': '#050505', 'bold': True})
    format_grand_total = writer.add_format({'bg_color': '#97ba9d', 'bold': True, 'font_color': '#050505'})
    format_default = writer.add_format({})
    
    for col_idx, col_name in enumerate(concat.columns):
        worksheet.write(0, col_idx, col_name, heading_format)
    
    for row_idx, row in enumerate(concat.iter_rows(), start=1):
        supplier, month = row[0], row[1]
        row_format = None

        if month == "total":
            row_format = format_total
        elif supplier == "Total":
            row_format = format_grand_total
        else:
            row_format = format_default

        for col_idx, cell_value in enumerate(row):
            worksheet.write(row_idx, col_idx, cell_value, row_format)

    writer.close()

C:\Users\D&B Trading\AppData\Local\Temp\ipykernel_16744\3327356542.py:4: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  .melt(id_vars=["Supplier", "month"], variable_name="Metric", value_name="Value")
